In [3]:
import subprocess

def run_prodigal_phage(inputfasta, out_gene, out_prot):
    to_run="prodigal -i "+inputfasta+" -o "+out_gene+" -a "+out_prot+" -p meta"
    subprocess.call(to_run.split(" "))
    
def run_blastp(inputfasta="input.fasta", output_file="blast.out", database="databast.fasta", evalue="0.001"):
    to_run="blastp -db "+database+" -query "+inputfasta+"-evalue "+evalue+" -outfmt 6 -out "+output_file
    subprocess.call(to_run.split(" "))

def run_formatdb(fastafile, protein="yes"):
    dbtype="prot"
    if protein=="no":
        dbtype="nucl"
    to_run="makeblastdb -in "+fastafile+" -dbtype "+dbtype
    subprocess.call(to_run.split(" "))
    
def run_formatudb(fastafile, databasefile="db.udb"):
    to_run="/home/sbiller/usearch7.0.1090_i86linux64 -makeudb_ublast "+fastafile+" -output "+databasefile
    subprocess.call(to_run.split(" "))
    
def run_ublastp(fastafile, out_file, udb, evalue):
    to_run="/home/sbiller/usearch7.0.1090_i86linux64 -ublast "+fastafile+" -db "+udb+" -evalue "+evalue+" -accel 0.5 -blast6out "+out_file+" -top_hit_only"
    subprocess.call(to_run.split(" "))
    
def run_trna_scan(input_file, output):
    if os.path.exists(output):
        os.remove(output)
    args=["tRNAscan-SE", "-o", output, "-G", "-D","-N", input_file]
    subprocess.call(args)
    print("tRNA scan of "+input_file+" is done!")

In [8]:
!scp ./databases/scripts/* jbrown@eofe4.mit.edu:/nobackup1/jbrown/annotation/databases/

format_blast_databases.py                     100%  272     0.3KB/s   00:00    
formatdb1.slurm                               100%  455     0.4KB/s   00:00    


In [9]:
!scp jbrown@eofe4.mit.edu:/nobackup1/jbrown/annotation/blasts/*.out ./

1.161.O.vsaclame.out                          100%   32KB  31.6KB/s   00:00    
1.161.O.vscogs_2003-2014.out                  100%  271KB 270.7KB/s   00:00    
1.161.O.vsCVP.out                             100%  260KB 259.9KB/s   00:00    
1.161.O.vsPfam.out                            100%  771KB 771.3KB/s   00:00    


In [11]:
!scp jbrown@eofe4.mit.edu:/nobackup1/jbrown/annotation/genes/1.161.O.* ./

1.161.O.gene                                  100%   54KB  54.0KB/s   00:00    


Below processing a prodigal gff file into a sequin table as described here: http://www.ncbi.nlm.nih.gov/genbank/genomesubmit-examples#fig2

In [60]:
gff="1.161.O.gene"

prod=open(gff).readlines()

digits=len(str(len(prod)/2))  ##for assigning a gene number with appropriate number of zeros preceding it

phage=gff.replace("gene","")
for i in range(2,len(prod)-1,2):
    loc=prod[i]
    info=prod[i+1]
    feature1="gene"
    feature2="CDS"
    if len(loc.split())==2:
        if "complement" in prod[i].split()[1]:
            start=loc.split("..")[1].replace(")\n","")
            stop=loc.split("(")[1].split("..")[0]
        else:
            start=loc.split()[1].split("..")[0]
            stop=loc.split()[1].split("..")[1].replace("\n","")
    number=info.split(";")[0].split("=")[2].split("_")[1]

    z="0"*(digits-len(number))
    
    locus_tag="nvp_"+phage+"_"+z+number
    #print(start+"\t"+stop+"\t"+feature1+"\n\t\t\tlocus_tag"+"\t"+locus_tag+"\n")
    #print(start+"\t"+stop+"\t"+feature2+"\n\t\t\tprotein_id"+"\t"+locus_tag+"\n")   

Processing aclame BLAST:

In [129]:
#modules for blast file processing

def get_locus_tag(line, digits, phage):
    query=line.split("\t")[0].split(" ")[0]
    number=query.split("_")[-1]
    z="0"*(digits-len(number))
    return "NVP"+phage.replace(".","")+"_"+z+number

In [54]:
blastout="1.161.O.vsaclame.out"
phage=blastout.split("v")[0]

blast=open(blastout).readlines()
for line in blast:
    print(line)

Vibriophage_1.161.O._10N.261.48.C5_2 # 388 # 1881 # 1 # ID=1_2;partial=00;start_type=ATG;rbs_motif=None;rbs_spacer=None;gc_cont=0.420	protein:vir:95253	33.7	454	264	11	20	473	94	510	1.4e-63	242.7

Vibriophage_1.161.O._10N.261.48.C5_2 # 388 # 1881 # 1 # ID=1_2;partial=00;start_type=ATG;rbs_motif=None;rbs_spacer=None;gc_cont=0.420	protein:proph:167812	33.7	457	247	18	20	465	37	448	2.3e-50	198.7

Vibriophage_1.161.O._10N.261.48.C5_15 # 10685 # 11692 # 1 # ID=1_15;partial=00;start_type=ATG;rbs_motif=AGGAG;rbs_spacer=5-10bp;gc_cont=0.438	protein:proph:170832	25.1	347	218	15	6	330	10	336	1.3e-12	72.8

Vibriophage_1.161.O._10N.261.48.C5_36 # 25242 # 25529 # -1 # ID=1_36;partial=00;start_type=ATG;rbs_motif=AGGAG;rbs_spacer=5-10bp;gc_cont=0.424	protein:plasmid:126053	40.2	82	43	2	10	87	19	98	1.4e-09	60.8

Vibriophage_1.161.O._10N.261.48.C5_36 # 25242 # 25529 # -1 # ID=1_36;partial=00;start_type=ATG;rbs_motif=AGGAG;rbs_spacer=5-10bp;gc_cont=0.424	protein:proph:185084	34.1	82	50	1	6	83	3	84	7.2e-

In [130]:
#set aclame dict for functions
aclame=open("./databases/DB_Info/aclame/aclame_proteins_all_0.4.tab").readlines()

aclame_dict={}
aclame_blast_dict={}

for line in aclame[2:-1]:
    protein=line.split("\t")[0]
    ncbi_ann=line.split("\t")[2]
    aclame_dict[protein]=ncbi_ann

#create a dict for BLAST results
blastout="1.161.O.vsaclame.out"
phage=blastout.split("v")[0]

blast=open(blastout).readlines()


for line in blast:
    locus_tag=get_locus_tag(line, digits, phage)
    hit=line.split("\t")[1]
    e=line.split("\t")[-2]
    bitscore=line.split("\t")[-1].replace("\n","")
    annotation=aclame_dict[hit]
    info=[hit, e, bitscore, annotation]
    if locus_tag not in aclame_blast_dict.keys():
        aclame_blast_dict[locus_tag]=info
    #print(locus_tag+"\t"+hit+"\t"+e+"\t"+bitscore+"\t"+aclame_dict[hit])

aclame_dict.clear()
print(aclame_blast_dict)

{'NVP1161O_131': ['protein:vir:5494', '5.5e-14', '76.3', 'hypothetical protein'], 'NVP1161O_228': ['protein:plasmid:152809', '1.6e-09', '62.8', 'hypothetical protein'], 'NVP1161O_015': ['protein:proph:170832', '1.3e-12', '72.8', 'hypothetical protein'], 'NVP1161O_224': ['protein:vir:96721', '9.5e-34', '142.1', 'putative phage-related protein'], 'NVP1161O_036': ['protein:plasmid:126053', '1.4e-09', '60.8', 'putative transcriptional regulator protein'], 'NVP1161O_053': ['protein:vir:95260', '1.5e-06', '51.6', 'Phage conserved protein'], 'NVP1161O_051': ['protein:plasmid:119499', '1.5e-39', '162.2', 'hypothetical protein'], 'NVP1161O_070': ['protein:vir:287', '8.5e-17', '87.0', 'putative tail fiber protein'], 'NVP1161O_150': ['protein:vir:94169', '7.3e-70', '264.2', 'putative ATP-dependent DNA helicase'], 'NVP1161O_158': ['protein:vir:680', '2.4e-20', '97.4', 'similar to other intron-encoded site specific endonucleases'], 'NVP1161O_196': ['protein:vir:99542', '8.2e-16', '81.6', 'hypotheti

In [131]:
cogs=open("./databases/DB_Info/COG/cog2003-2014.csv").readlines()
cogs2=open("./databases/DB_Info/COG/cognames2003-2014.tab").readlines()

cog_dict={}
cog_defs={}
cog_blast_dict={}

for line in cogs:
    gi=line.split(",")[0]
    cog=line.split(",")[6]
    cog_dict[gi]=cog
    
for line in cogs2:
    cog=line.split("\t")[0]
    func=line.split("\t")[2]
    cog_defs[cog]=func

blast=open("./1.161.O.vscogs_2003-2014.out").readlines()

for line in blast:
    locus_tag=get_locus_tag(line, digits, phage)
    hit=line.split("\t")[1]
    e=line.split("\t")[-2]
    bitscore=line.split("\t")[-1].replace("\n","")
    cog=cog_dict[(hit.split("|")[1])]
    func=cog_defs[cog]
    info=[hit, e, bitscore, cog, func]
    if locus_tag not in cog_blast_dict.keys():
        cog_blast_dict[locus_tag]=info

cog_dict.clear()
print cog_blast_dict

{'NVP1161O_067': ['gi|163855301|ref|YP_001629599.1|', '5.9e-41', '171.8', 'COG3299', 'Uncharacterized phage protein gp47/JayE\n'], 'NVP1161O_157': ['gi|88608725|ref|YP_506404.1|', '3.7e-19', '98.2', 'COG0208', 'Ribonucleotide reductase beta subunit, ferritin-like domain\n'], 'NVP1161O_146': ['gi|550890483|ref|YP_008664575.1|', '0.0001', '50.8', 'COG0740', 'ATP-dependent protease ClpP, protease subunit\n'], 'NVP1161O_208': ['gi|206895563|ref|YP_002247293.1|', '1.5e-06', '55.5', 'COG1186', 'Protein chain release factor B\n'], 'NVP1161O_166': ['gi|169825518|ref|YP_001695693.1|', '9.4e-10', '67.4', 'COG0338', 'Site-specific DNA-adenine methylase\n'], 'NVP1161O_061': ['gi|91777037|ref|YP_546793.1|', '2e-31', '140.6', 'COG3941', 'Phage tail tape-measure protein, controls tail length\n'], 'NVP1161O_211': ['gi|328955643|ref|YP_004372976.1|', '3.4e-05', '50.8', 'COG0236', 'Acyl carrier protein\n'], 'NVP1161O_168': ['gi|313674162|ref|YP_004052158.1|', '3.6e-12', '74.7', 'COG0262', 'Dihydrofolate

In [132]:
#now Pfam results:

pfams=open("./databases/DB_Info/PFam/Pfam-A.titles.txt").readlines()
pfams2=open("./databases/DB_Info/PFam/Pfam-A.clans.tsv").readlines()
pfam_dict={}
pfam2_dict={}
pfam_blast_dict={}

for line in pfams:
    seq=line.split(" ")[0].replace(">","")
    pfam=line.split(" ")[2].split(";")[0]
    pfam_dict[seq]=pfam
    
for line in pfams2:
    pfam=line.split("\t")[0]
    function=line.split("\t")[4]
    pfam2_dict[pfam]=function
    
blast=open("./1.161.O.vsPfam.out").readlines()

for line in blast:
    locus_tag=get_locus_tag(line, digits, phage)
    hit=line.split("\t")[1]
    e=line.split("\t")[-2]
    bitscore=line.split("\t")[-1].replace("\n","")
    pfam=pfam_dict[hit].split(".")[0]
    function=pfam2_dict[pfam].replace("\n","")
    info=[hit, e, bitscore, pfam, function]
    if locus_tag not in pfam_blast_dict.keys():
        pfam_blast_dict[locus_tag]=info
print(pfam_blast_dict)

pfam_dict.clear()
pfam2_dict.clear()

{'NVP1161O_208': ['Q1K271_DESAC/210-320', '3.1e-06', '56.6', 'PF00472', 'RF-1 domain'], 'NVP1161O_008': ['M4QU16_9VIRU/4-88', '4.4e-13', '80.5', 'PF10544', 'T5orf172 domain'], 'NVP1161O_015': ['R9U9M2_9VIRU/17-330', '5.4e-165', '585.5', 'PF03864', 'Phage major capsid protein E'], 'NVP1161O_012': ['F1D637_9CAUD/311-399', '1e-15', '89.7', 'PF10544', 'T5orf172 domain'], 'NVP1161O_011': ['M4M8X2_9VIRU/41-170', '3.1e-71', '273.1', 'PF14550', 'Putative phage serine protease XkdF'], 'NVP1161O_110': ['E5DSJ4_9CAUD/2-301', '6.4e-46', '189.9', 'PF03851', 'UV-endonuclease UvdE'], 'NVP1161O_036': ['R9U9N7_9VIRU/10-65', '1.5e-19', '100.5', 'PF01381', 'Helix-turn-helix'], 'NVP1161O_172': ['M4R2R0_9CAUD/136-379', '7.5e-127', '459.1', 'PF00149', 'Calcineurin-like phosphoesterase'], 'NVP1161O_052': ['F1D5V5_9CAUD/4-166', '3e-28', '130.2', 'PF09618', 'CRISPR-associated protein (Cas_Csy4)'], 'NVP1161O_050': ['F1D5V7_9CAUD/5-170', '1.4e-45', '188.3', 'PF09614', 'CRISPR-associated protein (Cas_Csy2)'], 'NV